In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import scipy.stats as stats
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler()
std = StandardScaler()
bay = linear_model.BayesianRidge()
rid = linear_model.Ridge(alpha=.5)
las = linear_model.Lasso()
lm = LinearRegression()

# Importing Dataset

In [2]:
df_train = pd.read_csv("Train_bigmart.csv")
df_test = pd.read_csv("Test_bigmart.csv")

In [3]:
df_train.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [4]:
df_train.shape

(8523, 12)

# Removing Item VIsibility with 0 value

In [6]:
len(df_train[df_train["Item_Visibility"] == 0])

526

In [7]:
df_train_new = df_train[df_train["Item_Visibility"] != 0]

# Transforming Item Visibility

In [9]:
df_train_new["Item_Visibility"] = np.log(df_train_new["Item_Visibility"])

F:\Anaconda_Installation\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


# New Dataset

In [10]:
df_train_new.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,-4.132215,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,-3.948780,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,-4.088756,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
6,FDO10,13.65,Regular,-4.362923,Snack Foods,57.6588,OUT013,1987,High,Tier 3,Supermarket Type1,343.5528
7,FDP10,NaN,Low Fat,-2.059875,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636


In [541]:
#df_train["Item_Visibility"] = mms.fit_transform(df_train.loc[:,["Item_Visibility"]])

# Removing Outliers of Sales Figure

In [11]:
df = df_train_new[np.abs(df_train_new.Item_Outlet_Sales-df_train_new.Item_Outlet_Sales.mean()) <= (3*df_train_new.Item_Outlet_Sales.std())]

# Transforming categorical variables

In [13]:
 for i in range(len(df)):
    if df.iloc[i,2] in ["Regular","reg"]:
         df.iloc[i,2] = 1
    else:
         df.iloc[i,2] = 2

In [14]:
df["Outlet_Location_Type"].unique()

array(['Tier 3', 'Tier 2', 'Tier 1'], dtype=object)

In [15]:
 for j in range(len(df)):
    if df.iloc[j,9] in ["Tier 1"]:
        df.iloc[j,9] = 1
    elif df.iloc[j,9] in ["Tier 2"]:
        df.iloc[j,9] = 2
    else:
        df.iloc[j,9] = 3

# Data after transformation

In [16]:
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
4254,FDN39,19.35,1,-2.210300,Meat,166.0816,OUT010,1998,NaN,3,Grocery Store,671.1264
2582,FDL46,20.35,2,-2.402624,Snack Foods,119.5466,OUT010,1998,NaN,3,Grocery Store,117.8466
2600,FDZ03,13.65,1,-2.025918,Dairy,188.0240,OUT010,1998,NaN,3,Grocery Store,745.6960
2620,FDQ14,9.27,2,-2.268964,Dairy,148.1050,OUT010,1998,NaN,3,Grocery Store,149.8050
7859,NCP06,20.70,2,-2.722817,Household,149.7366,OUT010,1998,NaN,3,Grocery Store,151.1366


In [20]:
items = df["Item_Identifier"].unique().tolist()

In [24]:
df_arr = np.array(df.loc[:,["Item_Identifier","Item_MRP"]])
list1=[]
list2 = []
for i in items:
    count=0
    sum1=0
    for j in df_arr:
        if(j[0]==i):
            count=count+1
            sum1=sum1+j[1]
    val=sum1/count
    list1.append(i)
    list2.append(val)

In [31]:
item_list = pd.DataFrame(list1)
item_avg_price = pd.DataFrame(list2)

In [30]:
df = df.sort_values(["Item_Identifier"])

In [68]:
for i in range(len(item_list)):
    for j in range(len(df)):
        if df.iloc[j,0] == item_list.iloc[i,0]:
            df.iloc[j,12] = item_avg_price.iloc[i,0]

In [73]:
df = df.drop(columns="Avg_MRP")

In [75]:
df["MRP_Diff"] = df["Avg_Item_Price"] - df["Item_MRP"]

In [76]:
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Avg_Item_Price,MRP_Diff
7467,DRA12,11.6,2,-3.191438,Soft Drinks,142.0154,OUT018,2009,Medium,3,Supermarket Type2,850.8924,141.9154,-0.1
1245,DRA12,11.6,2,-3.196336,Soft Drinks,142.3154,OUT013,1987,High,3,Supermarket Type1,2552.6772,141.9154,-0.4
8043,DRA12,11.6,2,-2.680410,Soft Drinks,143.0154,OUT010,1998,NaN,3,Grocery Store,283.6308,141.9154,-1.1
118,DRA12,11.6,2,-3.189863,Soft Drinks,140.3154,OUT017,2007,NaN,2,Supermarket Type1,2552.6772,141.9154,1.6
2879,DRA24,NaN,1,-2.660558,Soft Drinks,163.2868,OUT019,1985,Small,1,Grocery Store,491.3604,164.0868,0.8


# Final Dataframe

In [78]:
df.columns

Index(['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Visibility',
       'Item_Type', 'Item_MRP', 'Outlet_Identifier',
       'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type', 'Item_Outlet_Sales', 'Avg_Item_Price', 'MRP_Diff'],
      dtype='object')

In [84]:
df_final= df.loc[:,['Item_Identifier','Outlet_Identifier','Outlet_Type','Item_Fat_Content', 'Item_Visibility',
       'Item_Type','Outlet_Location_Type', 'MRP_Diff', 'Item_Outlet_Sales']]

In [85]:
df_final.head()

,Item_Identifier,Outlet_Identifier,Outlet_Type,Item_Fat_Content,Item_Visibility,Item_Type,Outlet_Location_Type,MRP_Diff,Item_Outlet_Sales
7467,DRA12,OUT018,Supermarket Type2,2,-3.191438,Soft Drinks,3,-0.1,850.8924
1245,DRA12,OUT013,Supermarket Type1,2,-3.196336,Soft Drinks,3,-0.4,2552.6772
8043,DRA12,OUT010,Grocery Store,2,-2.680410,Soft Drinks,3,-1.1,283.6308
118,DRA12,OUT017,Supermarket Type1,2,-3.189863,Soft Drinks,2,1.6,2552.6772
2879,DRA24,OUT019,Grocery Store,1,-2.660558,Soft Drinks,1,0.8,491.3604


# Creating Dummy Variables

In [86]:
df_final_1 = pd.get_dummies(df_final, columns=["Item_Fat_Content"])
df_final_2 = pd.get_dummies(df_final_1, columns=["Item_Type"])
df_final_3 = pd.get_dummies(df_final_2, columns=["Outlet_Location_Type"])

In [87]:
df_final_3.head()

,Item_Identifier,Outlet_Identifier,Outlet_Type,Item_Visibility,MRP_Diff,Item_Outlet_Sales,Item_Fat_Content_1,Item_Fat_Content_2,Item_Type_Baking Goods,Item_Type_Breads,...,Item_Type_Household,Item_Type_Meat,Item_Type_Others,Item_Type_Seafood,Item_Type_Snack Foods,Item_Type_Soft Drinks,Item_Type_Starchy Foods,Outlet_Location_Type_1,Outlet_Location_Type_2,Outlet_Location_Type_3
7467,DRA12,OUT018,Supermarket Type2,-3.191438,-0.1,850.8924,0,1,0,0,...,0,0,0,0,0,1,0,0,0,1
1245,DRA12,OUT013,Supermarket Type1,-3.196336,-0.4,2552.6772,0,1,0,0,...,0,0,0,0,0,1,0,0,0,1
8043,DRA12,OUT010,Grocery Store,-2.680410,-1.1,283.6308,0,1,0,0,...,0,0,0,0,0,1,0,0,0,1
118,DRA12,OUT017,Supermarket Type1,-3.189863,1.6,2552.6772,0,1,0,0,...,0,0,0,0,0,1,0,0,1,0
2879,DRA24,OUT019,Grocery Store,-2.660558,0.8,491.3604,1,0,0,0,...,0,0,0,0,0,1,0,1,0,0


In [100]:
df_final_3 = df_final_3.sort_values(["Outlet_Identifier"])

# Dividing dataframe in two parts

In [101]:
df_gr = df_final_3[df_final_3["Outlet_Type"] == "Grocery Store"]
df_su = df_final_3[df_final_3["Outlet_Type"] != "Grocery Store"]

In [102]:
df_gr.columns

Index(['Item_Identifier', 'Outlet_Identifier', 'Outlet_Type',
       'Item_Visibility', 'MRP_Diff', 'Item_Outlet_Sales',
       'Item_Fat_Content_1', 'Item_Fat_Content_2', 'Item_Type_Baking Goods',
       'Item_Type_Breads', 'Item_Type_Breakfast', 'Item_Type_Canned',
       'Item_Type_Dairy', 'Item_Type_Frozen Foods',
       'Item_Type_Fruits and Vegetables', 'Item_Type_Hard Drinks',
       'Item_Type_Health and Hygiene', 'Item_Type_Household', 'Item_Type_Meat',
       'Item_Type_Others', 'Item_Type_Seafood', 'Item_Type_Snack Foods',
       'Item_Type_Soft Drinks', 'Item_Type_Starchy Foods',
       'Outlet_Location_Type_1', 'Outlet_Location_Type_2',
       'Outlet_Location_Type_3'],
      dtype='object')

In [106]:
df_gr = df_gr.loc[:,['Item_Identifier','Outlet_Type','Outlet_Identifier',
       'Item_Visibility', 'MRP_Diff',
       'Item_Fat_Content_1', 'Item_Fat_Content_2', 'Item_Type_Baking Goods',
       'Item_Type_Breads', 'Item_Type_Breakfast', 'Item_Type_Canned',
       'Item_Type_Dairy', 'Item_Type_Frozen Foods',
       'Item_Type_Fruits and Vegetables', 'Item_Type_Hard Drinks',
       'Item_Type_Health and Hygiene', 'Item_Type_Household', 'Item_Type_Meat',
       'Item_Type_Others', 'Item_Type_Seafood', 'Item_Type_Snack Foods',
       'Item_Type_Soft Drinks', 'Item_Type_Starchy Foods',
       'Outlet_Location_Type_1', 'Outlet_Location_Type_2',
       'Outlet_Location_Type_3','Item_Outlet_Sales']]
df_su = df_su.loc[:,['Item_Identifier','Outlet_Type','Outlet_Identifier',
       'Item_Visibility', 'MRP_Diff',
       'Item_Fat_Content_1', 'Item_Fat_Content_2', 'Item_Type_Baking Goods',
       'Item_Type_Breads', 'Item_Type_Breakfast', 'Item_Type_Canned',
       'Item_Type_Dairy', 'Item_Type_Frozen Foods',
       'Item_Type_Fruits and Vegetables', 'Item_Type_Hard Drinks',
       'Item_Type_Health and Hygiene', 'Item_Type_Household', 'Item_Type_Meat',
       'Item_Type_Others', 'Item_Type_Seafood', 'Item_Type_Snack Foods',
       'Item_Type_Soft Drinks', 'Item_Type_Starchy Foods',
       'Outlet_Location_Type_1', 'Outlet_Location_Type_2',
       'Outlet_Location_Type_3','Item_Outlet_Sales']]

In [107]:
df_su.head()

,Item_Identifier,Outlet_Type,Outlet_Identifier,Item_Visibility,MRP_Diff,Item_Fat_Content_1,Item_Fat_Content_2,Item_Type_Baking Goods,Item_Type_Breads,Item_Type_Breakfast,...,Item_Type_Meat,Item_Type_Others,Item_Type_Seafood,Item_Type_Snack Foods,Item_Type_Soft Drinks,Item_Type_Starchy Foods,Outlet_Location_Type_1,Outlet_Location_Type_2,Outlet_Location_Type_3,Item_Outlet_Sales
4155,FDU02,Supermarket Type1,OUT013,-2.278613,-0.900000,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,5725.8800
1376,FDH35,Supermarket Type1,OUT013,-2.810108,1.700000,0,1,0,0,0,...,0,0,0,0,0,1,0,0,1,1151.1682
2869,FDT55,Supermarket Type1,OUT013,-3.132266,0.114286,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2840.3028
7503,FDI28,Supermarket Type1,OUT013,-3.638194,-0.716667,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,1743.0644
4947,NCB55,Supermarket Type1,OUT013,-1.829279,-0.725000,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,296.2810


In [108]:
df_train_gr = df_gr.iloc[:,2:27]
df_train_su = df_su.iloc[:,2:27]

In [109]:
df_train_gr.head()

,Outlet_Identifier,Item_Visibility,MRP_Diff,Item_Fat_Content_1,Item_Fat_Content_2,Item_Type_Baking Goods,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,Item_Type_Dairy,...,Item_Type_Meat,Item_Type_Others,Item_Type_Seafood,Item_Type_Snack Foods,Item_Type_Soft Drinks,Item_Type_Starchy Foods,Outlet_Location_Type_1,Outlet_Location_Type_2,Outlet_Location_Type_3,Item_Outlet_Sales
1754,OUT010,-1.586146,-0.414286,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,202.4032
7362,OUT010,-2.437401,-1.540000,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,369.5190
3025,OUT010,-1.855100,-0.216667,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,284.9624
6948,OUT010,-3.559848,-0.700000,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,386.1640
851,OUT010,-1.917503,-2.200000,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,547.2876


# Dividing further into different store

In [110]:
out10_g = []
out13_s = []
out17_s = []
out18_s = []
out19_g = []
out27_s = []
out35_s = []
out45_s = []
out46_s = []
out49_s = []

In [111]:
for i in range(len(df_train_su)):
        if df_train_su.iloc[i,0] in ["OUT013"]:
            out13_s.append(df_train_su.iloc[i,:])
        elif df_train_su.iloc[i,0] in ["OUT017"]:
            out17_s.append(df_train_su.iloc[i,:])
        elif df_train_su.iloc[i,0] in ["OUT018"]:
            out18_s.append(df_train_su.iloc[i,:])
        elif df_train_su.iloc[i,0] in ["OUT027"]:
            out27_s.append(df_train_su.iloc[i,:])
        elif df_train_su.iloc[i,0] in ["OUT035"]:
            out35_s.append(df_train_su.iloc[i,:])
        elif df_train_su.iloc[i,0] in ["OUT045"]:
            out45_s.append(df_train_su.iloc[i,:])
        elif df_train_su.iloc[i,0] in ["OUT046"]:
            out46_s.append(df_train_su.iloc[i,:])
        else:
            out49_s.append(df_train_su.iloc[i,:])

In [112]:
for j in range(len(df_train_gr)):
    if df_train_gr.iloc[j,0] in ["OUT010"]:
        out10_g.append(df_train_gr.iloc[j,:])
    elif df_train_gr.iloc[j,0] in ["OUT019"]:
        out19_g.append(df_train_gr.iloc[j,:])

In [113]:
out10_dum = pd.DataFrame(out10_g)
out13_dum = pd.DataFrame(out13_s)
out17_dum = pd.DataFrame(out17_s)
out18_dum = pd.DataFrame(out18_s)
out19_dum = pd.DataFrame(out19_g)
out27_dum = pd.DataFrame(out27_s)
out35_dum = pd.DataFrame(out35_s)
out45_dum = pd.DataFrame(out45_s)
out46_dum = pd.DataFrame(out46_s)
out49_dum = pd.DataFrame(out49_s)

In [114]:
out10_dum.head()

,Outlet_Identifier,Item_Visibility,MRP_Diff,Item_Fat_Content_1,Item_Fat_Content_2,Item_Type_Baking Goods,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,Item_Type_Dairy,...,Item_Type_Meat,Item_Type_Others,Item_Type_Seafood,Item_Type_Snack Foods,Item_Type_Soft Drinks,Item_Type_Starchy Foods,Outlet_Location_Type_1,Outlet_Location_Type_2,Outlet_Location_Type_3,Item_Outlet_Sales
1754,OUT010,-1.586146,-0.414286,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,202.4032
7362,OUT010,-2.437401,-1.540000,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,369.5190
3025,OUT010,-1.855100,-0.216667,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,284.9624
6948,OUT010,-3.559848,-0.700000,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,386.1640
851,OUT010,-1.917503,-2.200000,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,547.2876


In [128]:
out10_dum.shape

(526, 25)

In [130]:
out10_dum.iloc[:,1:24].head()

,Item_Visibility,MRP_Diff,Item_Fat_Content_1,Item_Fat_Content_2,Item_Type_Baking Goods,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,Item_Type_Dairy,Item_Type_Frozen Foods,...,Item_Type_Household,Item_Type_Meat,Item_Type_Others,Item_Type_Seafood,Item_Type_Snack Foods,Item_Type_Soft Drinks,Item_Type_Starchy Foods,Outlet_Location_Type_1,Outlet_Location_Type_2,Outlet_Location_Type_3
1754,-1.586146,-0.414286,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7362,-2.437401,-1.540000,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3025,-1.855100,-0.216667,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6948,-3.559848,-0.700000,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
851,-1.917503,-2.200000,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1


# Splitting Datasets into training and test

In [134]:
out10_train_x, out10_test_x, out10_train_y, out10_test_y = train_test_split(out10_dum.iloc[:,1:24], out10_dum.loc[:,["Item_Outlet_Sales"]], test_size=0.2)
out13_train_x, out13_test_x, out13_train_y, out13_test_y = train_test_split(out13_dum.iloc[:,1:24], out13_dum.loc[:,["Item_Outlet_Sales"]], test_size=0.2)
out17_train_x, out17_test_x, out17_train_y, out17_test_y = train_test_split(out17_dum.iloc[:,1:24], out17_dum.loc[:,["Item_Outlet_Sales"]], test_size=0.2)
out18_train_x, out18_test_x, out18_train_y, out18_test_y = train_test_split(out18_dum.iloc[:,1:24], out18_dum.loc[:,["Item_Outlet_Sales"]], test_size=0.2)
out19_train_x, out19_test_x, out19_train_y, out19_test_y = train_test_split(out19_dum.iloc[:,1:24], out19_dum.loc[:,["Item_Outlet_Sales"]], test_size=0.2)
out27_train_x, out27_test_x, out27_train_y, out27_test_y = train_test_split(out27_dum.iloc[:,1:24], out27_dum.loc[:,["Item_Outlet_Sales"]], test_size=0.2)
out35_train_x, out35_test_x, out35_train_y, out35_test_y = train_test_split(out35_dum.iloc[:,1:24], out35_dum.loc[:,["Item_Outlet_Sales"]], test_size=0.2)
out45_train_x, out45_test_x, out45_train_y, out45_test_y = train_test_split(out45_dum.iloc[:,1:24], out45_dum.loc[:,["Item_Outlet_Sales"]], test_size=0.2)
out46_train_x, out46_test_x, out46_train_y, out46_test_y = train_test_split(out46_dum.iloc[:,1:24], out46_dum.loc[:,["Item_Outlet_Sales"]], test_size=0.2)
out49_train_x, out49_test_x, out49_train_y, out49_test_y = train_test_split(out49_dum.iloc[:,1:24], out49_dum.loc[:,["Item_Outlet_Sales"]], test_size=0.2)

In [135]:
df_test_y =pd.concat([out10_test_y, 
            out13_test_y, 
            out17_test_y, 
            out18_test_y, 
            out19_test_y, 
            out27_test_y, 
            out35_test_y, 
            out45_test_y, 
            out46_test_y, 
            out49_test_y], axis = 0)

# Linear Regression

In [137]:
lm.fit(out13_train_x, out13_train_y)
lm.fit(out17_train_x, out17_train_y)
lm.fit(out18_train_x, out18_train_y)
lm.fit(out27_train_x, out27_train_y)
lm.fit(out35_train_x, out35_train_y)
lm.fit(out45_train_x, out45_train_y)
lm.fit(out46_train_x, out46_train_y)
lm.fit(out49_train_x, out49_train_y)
lm.fit(out10_train_x, out10_train_y)
lm.fit(out19_train_x, out19_train_y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [138]:
out13_dum_pred_lm = lm.predict(out13_test_x)
out17_dum_pred_lm = lm.predict(out17_test_x)
out18_dum_pred_lm = lm.predict(out18_test_x)
out27_dum_pred_lm = lm.predict(out27_test_x)
out35_dum_pred_lm = lm.predict(out35_test_x)
out45_dum_pred_lm = lm.predict(out45_test_x)
out46_dum_pred_lm = lm.predict(out46_test_x)
out49_dum_pred_lm = lm.predict(out49_test_x)
out10_dum_pred_lm = lm.predict(out10_test_x)
out19_dum_pred_lm = lm.predict(out19_test_x)

In [139]:
out13_dum_pred_lm = pd.DataFrame(out13_dum_pred_lm)
out17_dum_pred_lm = pd.DataFrame(out17_dum_pred_lm)
out18_dum_pred_lm = pd.DataFrame(out18_dum_pred_lm)
out27_dum_pred_lm = pd.DataFrame(out27_dum_pred_lm)
out35_dum_pred_lm = pd.DataFrame(out35_dum_pred_lm)
out45_dum_pred_lm = pd.DataFrame(out45_dum_pred_lm)
out46_dum_pred_lm = pd.DataFrame(out46_dum_pred_lm)
out49_dum_pred_lm = pd.DataFrame(out49_dum_pred_lm)
out10_dum_pred_lm = pd.DataFrame(out10_dum_pred_lm)
out19_dum_pred_lm = pd.DataFrame(out19_dum_pred_lm)

In [140]:
df_pred_lm = pd.concat([out10_dum_pred_lm,
                out13_dum_pred_lm, 
                out17_dum_pred_lm, 
                out18_dum_pred_lm, 
                out19_dum_pred_lm, 
                out27_dum_pred_lm,
                out35_dum_pred_lm, 
                out45_dum_pred_lm, 
                out46_dum_pred_lm, 
                out49_dum_pred_lm], axis = 0)

In [141]:
rms_lm = sqrt(mean_squared_error(df_test_y, df_pred_lm))

In [142]:
rms_lm

2316.7166661956835

# Bayesian Regressor

In [143]:
bay.fit(out13_train_x, out13_train_y)
bay.fit(out17_train_x, out17_train_y)
bay.fit(out18_train_x, out18_train_y)
bay.fit(out27_train_x, out27_train_y)
bay.fit(out35_train_x, out35_train_y)
bay.fit(out45_train_x, out45_train_y)
bay.fit(out46_train_x, out46_train_y)
bay.fit(out49_train_x, out49_train_y)
bay.fit(out10_train_x, out10_train_y)
bay.fit(out19_train_x, out19_train_y)

F:\Anaconda_Installation\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
F:\Anaconda_Installation\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
F:\Anaconda_Installation\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
F:\Anaconda_Installation\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, compute_score=False, copy_X=True,
       fit_intercept=True, lambda_1=1e-06, lambda_2=1e-06, n_iter=300,
       normalize=False, tol=0.001, verbose=False)

In [144]:
out13_dum_pred_bay = bay.predict(out13_test_x)
out17_dum_pred_bay = bay.predict(out17_test_x)
out18_dum_pred_bay = bay.predict(out18_test_x)
out27_dum_pred_bay = bay.predict(out27_test_x)
out35_dum_pred_bay = bay.predict(out35_test_x)
out45_dum_pred_bay = bay.predict(out45_test_x)
out46_dum_pred_bay = bay.predict(out46_test_x)
out49_dum_pred_bay = bay.predict(out49_test_x)
out10_dum_pred_bay = bay.predict(out10_test_x)
out19_dum_pred_bay = bay.predict(out19_test_x)

In [145]:
out13_dum_pred_lm = pd.DataFrame(out13_dum_pred_lm)
out17_dum_pred_lm = pd.DataFrame(out17_dum_pred_lm)
out18_dum_pred_lm = pd.DataFrame(out18_dum_pred_lm)
out27_dum_pred_lm = pd.DataFrame(out27_dum_pred_lm)
out35_dum_pred_lm = pd.DataFrame(out35_dum_pred_lm)
out45_dum_pred_lm = pd.DataFrame(out45_dum_pred_lm)
out46_dum_pred_lm = pd.DataFrame(out46_dum_pred_lm)
out49_dum_pred_lm = pd.DataFrame(out49_dum_pred_lm)
out10_dum_pred_lm = pd.DataFrame(out10_dum_pred_lm)
out19_dum_pred_lm = pd.DataFrame(out19_dum_pred_lm)

In [146]:
out13_dum_pred_bay = pd.DataFrame(out13_dum_pred_bay)
out17_dum_pred_bay = pd.DataFrame(out17_dum_pred_bay)
out18_dum_pred_bay = pd.DataFrame(out18_dum_pred_bay)
out27_dum_pred_bay = pd.DataFrame(out27_dum_pred_bay)
out35_dum_pred_bay = pd.DataFrame(out35_dum_pred_bay)
out45_dum_pred_bay = pd.DataFrame(out45_dum_pred_bay)
out46_dum_pred_bay = pd.DataFrame(out46_dum_pred_bay)
out49_dum_pred_bay = pd.DataFrame(out49_dum_pred_bay)
out10_dum_pred_bay = pd.DataFrame(out10_dum_pred_bay)
out19_dum_pred_bay = pd.DataFrame(out19_dum_pred_bay)

In [147]:
df_pred_bay = pd.concat([out10_dum_pred_bay,
                out13_dum_pred_bay, 
                out17_dum_pred_bay, 
                out18_dum_pred_bay, 
                out19_dum_pred_bay, 
                out27_dum_pred_bay,
                out35_dum_pred_bay, 
                out45_dum_pred_bay, 
                out46_dum_pred_bay, 
                out49_dum_pred_bay], axis = 0)

In [148]:
rms_bay = sqrt(mean_squared_error(df_test_y, df_pred_bay))

In [149]:
rms_bay

2316.833017804517

# Ridge Regressor

In [150]:
rid.fit(out13_train_x, out13_train_y)
rid.fit(out17_train_x, out17_train_y)
rid.fit(out18_train_x, out18_train_y)
rid.fit(out27_train_x, out27_train_y)
rid.fit(out35_train_x, out35_train_y)
rid.fit(out45_train_x, out45_train_y)
rid.fit(out46_train_x, out46_train_y)
rid.fit(out49_train_x, out49_train_y)
rid.fit(out10_train_x, out10_train_y)
rid.fit(out19_train_x, out19_train_y)

Ridge(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [151]:
out13_dum_pred_rid = rid.predict(out13_test_x)
out17_dum_pred_rid = rid.predict(out17_test_x)
out18_dum_pred_rid = rid.predict(out18_test_x)
out27_dum_pred_rid = rid.predict(out27_test_x)
out35_dum_pred_rid = rid.predict(out35_test_x)
out45_dum_pred_rid = rid.predict(out45_test_x)
out46_dum_pred_rid = rid.predict(out46_test_x)
out49_dum_pred_rid = rid.predict(out49_test_x)
out10_dum_pred_rid = rid.predict(out10_test_x)
out19_dum_pred_rid = rid.predict(out19_test_x)

In [152]:
out13_dum_pred_rid = pd.DataFrame(out13_dum_pred_rid)
out17_dum_pred_rid = pd.DataFrame(out17_dum_pred_rid)
out18_dum_pred_rid = pd.DataFrame(out18_dum_pred_rid)
out27_dum_pred_rid = pd.DataFrame(out27_dum_pred_rid)
out35_dum_pred_rid = pd.DataFrame(out35_dum_pred_rid)
out45_dum_pred_rid = pd.DataFrame(out45_dum_pred_rid)
out46_dum_pred_rid = pd.DataFrame(out46_dum_pred_rid)
out49_dum_pred_rid = pd.DataFrame(out49_dum_pred_rid)
out10_dum_pred_rid = pd.DataFrame(out10_dum_pred_rid)
out19_dum_pred_rid = pd.DataFrame(out19_dum_pred_rid)

In [153]:
df_pred_rid = pd.concat([out10_dum_pred_rid,
                out13_dum_pred_rid, 
                out17_dum_pred_rid, 
                out18_dum_pred_rid, 
                out19_dum_pred_rid, 
                out27_dum_pred_rid,
                out35_dum_pred_rid, 
                out45_dum_pred_rid, 
                out46_dum_pred_rid, 
                out49_dum_pred_rid], axis = 0)

In [154]:
rms_rid = sqrt(mean_squared_error(df_test_y, df_pred_rid))

In [155]:
rms_rid

2316.4229966333864

# Lasso

In [157]:
las.fit(out13_train_x, out13_train_y)
las.fit(out17_train_x, out17_train_y)
las.fit(out18_train_x, out18_train_y)
las.fit(out27_train_x, out27_train_y)
las.fit(out35_train_x, out35_train_y)
las.fit(out45_train_x, out45_train_y)
las.fit(out46_train_x, out46_train_y)
las.fit(out49_train_x, out49_train_y)
las.fit(out10_train_x, out10_train_y)
las.fit(out19_train_x, out19_train_y)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [159]:
out13_dum_pred_las = las.predict(out13_test_x)
out17_dum_pred_las = las.predict(out17_test_x)
out18_dum_pred_las = las.predict(out18_test_x)
out27_dum_pred_las = las.predict(out27_test_x)
out35_dum_pred_las = las.predict(out35_test_x)
out45_dum_pred_las = las.predict(out45_test_x)
out46_dum_pred_las = las.predict(out46_test_x)
out49_dum_pred_las = las.predict(out49_test_x)
out10_dum_pred_las = las.predict(out10_test_x)
out19_dum_pred_las = las.predict(out19_test_x)

In [160]:
out13_dum_pred_las = pd.DataFrame(out13_dum_pred_las)
out17_dum_pred_las = pd.DataFrame(out17_dum_pred_las)
out18_dum_pred_las = pd.DataFrame(out18_dum_pred_las)
out27_dum_pred_las = pd.DataFrame(out27_dum_pred_las)
out35_dum_pred_las = pd.DataFrame(out35_dum_pred_las)
out45_dum_pred_las = pd.DataFrame(out45_dum_pred_las)
out46_dum_pred_las = pd.DataFrame(out46_dum_pred_las)
out49_dum_pred_las = pd.DataFrame(out49_dum_pred_las)
out10_dum_pred_las = pd.DataFrame(out10_dum_pred_las)
out19_dum_pred_las = pd.DataFrame(out19_dum_pred_las)

In [162]:
df_pred_las = pd.concat([out10_dum_pred_las,
                out13_dum_pred_las, 
                out17_dum_pred_las, 
                out18_dum_pred_las, 
                out19_dum_pred_las, 
                out27_dum_pred_las,
                out35_dum_pred_las, 
                out45_dum_pred_las, 
                out46_dum_pred_las, 
                out49_dum_pred_las], axis = 0)

In [163]:
rms_las = sqrt(mean_squared_error(df_test_y, df_pred_las))

In [164]:
rms_las

2315.870291596196